# EDA & Cleaning

## Setup & Load

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Path to the WHO Excel
xlsx_path = Path("who_ambient_air_quality_database_version_2024_(v6.1).xlsx")

# Check available sheets
xls = pd.ExcelFile(xlsx_path)
print(xls.sheet_names)   

# Load the main data sheet
df = pd.read_excel(xlsx_path, sheet_name="Update 2024 (V6.1)")

df.shape
df

['Readme', 'Metadata', 'Update 2024 (V6.1)']


,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,4_Eur,ESP,Spain,A Coruna/ESP,2013.0,"V4.0 (2018), V4.0 (2018), V4.0 (2018), V4.0 (2...",23.238,11.491,28.841,87.0,46.0,93.0,"Urban, Urban, Suburban",NaN,NaN,246146.0,"manual, manual, manual, manual",43.367900,-8.418571,1
1,4_Eur,ESP,Spain,A Coruna/ESP,2014.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023)",27.476,15.878,19.575,96.0,88.0,95.0,"Urban, Urban, Suburban",NaN,NaN,247604.0,NaN,43.368033,-8.418233,1
2,4_Eur,ESP,Spain,A Coruna/ESP,2015.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",25.515,14.004,22.731,98.0,71.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
3,4_Eur,ESP,Spain,A Coruna/ESP,2016.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",23.057,13.160,20.204,98.0,98.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
4,4_Eur,ESP,Spain,A Coruna/ESP,2017.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",26.849,14.114,21.543,97.0,97.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40093,6_Wpr,KOR,Republic of Korea,경기도/KOR,2017.0,V5.0 (2022),57.335,36.457,0.029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.337200,126.724100,1
40094,6_Wpr,KOR,Republic of Korea,경기도/KOR,2018.0,V5.0 (2022),50.838,31.586,0.027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.337200,126.724100,1
40095,6_Wpr,KOR,Republic of Korea,경기도/KOR,2019.0,V5.0 (2022),55.568,31.013,0.028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.337200,126.724100,1
40096,6_Wpr,CHN,China,虎英公园北/CHN,2018.0,V5.0 (2022),NaN,30.649,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,23.012778,113.794444,1


In [2]:
# Strip whitespace in text columns
for col in df.select_dtypes(include="object"):
    df[col] = df[col].astype(str).str.strip().replace({"nan": np.nan})

# Force numeric for key columns
num_cols = ["year","pm10_concentration","pm25_concentration","no2_concentration",
            "pm10_tempcov","pm25_tempcov","no2_tempcov",
            "population","latitude","longitude"]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

In [3]:
df = df.dropna(subset=["year","country_name"]).copy()
df["city"] = df["city"].fillna("Unknown")

In [4]:
df["unit"] = "µg/m³"
df["source"] = "WHO Ambient Air Quality Database (v6.1, Jan 2024)"
df["dataset_version"] = "2024_v6.1"

In [5]:
id_cols = ["who_region","iso3","country_name","city","year","version",
           "type_of_stations","reference","web_link","population","population_source",
           "latitude","longitude","who_ms","unit","source","dataset_version"]

value_map = {
    "pm25": {"conc":"pm25_concentration","tempcov":"pm25_tempcov"},
    "pm10": {"conc":"pm10_concentration","tempcov":"pm10_tempcov"},
    "no2":  {"conc":"no2_concentration", "tempcov":"no2_tempcov"},
}

long_frames = []
for pollutant, cols in value_map.items():
    tmp = df[id_cols].copy()
    tmp["pollutant"] = pollutant
    tmp["concentration"] = df[cols["conc"]]
    tmp["temporal_coverage_pct"] = df[cols["tempcov"]]
    long_frames.append(tmp)

df_long = pd.concat(long_frames, ignore_index=True)

# Keep only rows with actual concentration
df_long = df_long[~df_long["concentration"].isna()]

## Missingness Audit

In [6]:
missing = df.isna().mean().sort_values(ascending=False).to_frame("pct_missing").assign(pct_missing=lambda x: (x*100).round(2))
missing.head(15)

,pct_missing
web_link,95.54
reference,83.41
pm25_tempcov,58.62
population_source,54.86
pm25_concentration,45.81
pm10_tempcov,44.13
population,44.06
type_of_stations,41.81
no2_tempcov,41.63
no2_concentration,32.82


In [7]:
keys = ["who_region","iso3","country_name","city","pollutant","type_of_stations","year"]
uniques = {k: df[k].nunique(dropna=True) for k in keys if k in df.columns}
pd.Series(uniques, name="n_unique").to_frame()

,n_unique
who_region,7
iso3,123
country_name,124
city,7182
type_of_stations,325
year,13


## Duplicate Check

In [8]:
pk_cols = ["iso3","city","year"]
dups = (df[pk_cols].assign(n=1)
        .groupby(pk_cols, as_index=False)["n"]
        .sum()
        .query("n > 1"))
print("Duplicate city-year groups:", len(dups))
dups.head(10)

Duplicate city-year groups: 0


,iso3,city,year,n


## Geospatial Sanity Checks

In [9]:
lat_ok = df["latitude"].between(-90, 90, inclusive="both").fillna(True)
lon_ok = df["longitude"].between(-180, 180, inclusive="both").fillna(True)
print("Out-of-range lat:", (~lat_ok).sum(), "| Out-of-range lon:", (~lon_ok).sum())

df.loc[~lat_ok, "latitude"] = np.nan
df.loc[~lon_ok, "longitude"] = np.nan

Out-of-range lat: 0 | Out-of-range lon: 0


## 📊 Basic Descriptives

In [10]:
num_desc = df.select_dtypes(include=[np.number]).describe().T
num_desc

,count,mean,std,min,25%,50%,75%,max
year,40095.0,2016.232672,3.116678e+00,2010.000000,2014.00000,2016.000000,2019.000000,2.022000e+03
pm10_concentration,28672.0,30.016375,2.954857e+01,0.730000,16.26475,21.335500,30.515500,5.400000e+02
pm25_concentration,21729.0,19.129778,1.748962e+01,1.000000,8.60300,12.710000,23.000000,4.364400e+02
no2_concentration,26934.0,18.861620,2.584506e+01,0.002000,10.16325,16.700000,24.911500,3.670314e+03
pm10_tempcov,22403.0,88.732134,2.027727e+01,0.000000,90.00000,96.000000,99.000000,1.000000e+02
pm25_tempcov,16590.0,84.124533,2.681910e+01,0.000000,86.00000,96.000000,99.000000,1.000000e+02
no2_tempcov,23402.0,88.092214,2.314964e+01,0.000000,92.00000,96.000000,98.000000,1.000000e+02
population,22430.0,756416.161034,2.122104e+06,5.000000,53006.00000,163682.000000,590032.000000,3.739313e+07
latitude,40095.0,40.026037,1.630879e+01,-53.158316,37.13880,43.823955,49.163609,6.967750e+01
longitude,40095.0,10.593340,5.777409e+01,-159.366240,-1.25890,10.512200,23.226400,1.784500e+02


In [11]:
out_dir = Path("who_aq_clean")
out_dir.mkdir(exist_ok=True)

# Wide (similar to WHO original)
df.to_csv(out_dir/"who_aq_2024_wide_clean.csv", index=False)

# Long tidy format
df_long.to_csv(out_dir/"who_aq_2024_long_clean.csv", index=False)

In [12]:
summary = (df_long
           .groupby(["country_name","iso3","who_region","year","pollutant"], as_index=False)
           .agg(avg_concentration=("concentration","mean"),
                cities_reporting=("city","nunique"),
                pop_sum=("population","sum")))

summary.to_csv(out_dir/"who_aq_2024_country_year_summary.csv", index=False)